# Data Preprocessing for Machine Learning

##### by Márk Kereszty
*Note: This Jupyter Notebook is optimised for the **Design and development of an IoT-based data collection/data analysis system** thesis project!*



To use the collected data for model training in the field of Machine Learning, we need to make some preparations first. These are so-called preprocessing steps, that can vary based on the input and the desired output. This notebook contains the following steps:
- converting .csv dataset to Pandas DataFrame
- dropping irrelevant columns
- grouping measurements based on the timestamps
- converting strings to floats and extracting necessary information
- filling in the missing values
- scaling the dataset for optimal value range

These steps can be built into a preprocessing pipeline in scikit-learn, but for now let's go over each one separately.

### Loading the Dataset

First lets select the data you want to process. This should be a file with *.csv* extension, downloaded from the database via the [OpenDAQ dashboard]().

In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from datetime import timedelta
import ast
import numpy as np

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()

Now that you have selected the source, the next step is to load it into a [Pandas](https://pandas.pydata.org/) DataFrame. 

In [2]:

df = pd.read_csv(file_path)
print(f"Loaded {len(df.index)} records across {len(df['variable'].unique())} variables.")


Loaded 228 records across 12 variables.


### Restructuring the Data

Since *_result, table, _measurement, aspect, org, sensor, student and topic* columns don't contain relevant information in the scope of ML, we can go ahead and drop them.
The next step is to organize the data. With the help of the timestamps we can determine which records are correlated. After grouping them by timestamp we can combine each type of data into a single row. This way the measurements ar logically organised and more manageable. Finally we can also drop the columns ending in *_fft*, the PSD columns hold more value in regards of vibration analysis.


In [3]:
labels = ['result', 'table', '_measurement', 'aspect', 'org', 'sensor', 'student', 'topic']
df.drop(labels=labels, axis=1, inplace=True)

# grouping by timestamp
df['_time'] = pd.to_datetime(df['_time'])
df = df.sort_values('_time')

grouped_measurements = []
current_group = []
current_time = None

for index, row in df.iterrows():
    if current_time is None:
        current_time = row['_time']
        current_group.append(row)
    elif (row['_time'] - current_time) <= timedelta(seconds=1):
        current_group.append(row)
    else:
        grouped_measurements.append(current_group)
        current_group = [row]
        current_time = row['_time']

# appending the last group
grouped_measurements.append(current_group)

# new dataframe from the grouped values
result_data = {'_time': [group[0]['_time'] for group in grouped_measurements]}
for variable in df['variable'].unique():
    result_data[variable] = [next((row['_value'] for row in group if row['variable'] == variable), None) for group in grouped_measurements]

df = pd.DataFrame(result_data)

# droping fft columns
for column in df.columns:
    if 'fft' in column:
        df.drop(labels=column, axis=1, inplace=True)


### Strings to Numbers

Many values are stored as strings inside InfluxDB, since they are lists. First we need to convert them back to lists of floats using the *ast* library. For the phase currents we can calculate the sum and store them instead of separate current values. The RMS values are also strings because they are part of the *lathe_analytics* measurement group and the other two variables (fft and psd) are strings. These only need to be converted back to float.

Lastly we have the PSD data. The method I choose here is to only store the *n* highest power values with their corresponding frequencies, for each axis. This way we can recognise patterns while keeping the dataset manageable.

In [4]:
# converting the cur column to sum of currents
df['cur'] = df['cur'].apply(lambda x: sum(ast.literal_eval(x)) if x is not None else None)

# converting rms column from str to float
for column in df.columns:
    if 'rms' in column:
        df[column] = df[column].apply(lambda x: float(x) if x is not None else None)



def process_psd(original_df, column, count=10):
    column_names = [f'{column[:4]}_pow_{i+1}' for i in range(count)]
    column_names += [f'{column[:4]}_freq_{i+1}' for i in range(count)]
    df = pd.DataFrame(columns=column_names)

    for index, row in original_df.iterrows():
        try:
            ls = ast.literal_eval(row[column])
        
            sorted_data = sorted(zip(ls[1], ls[0]), key=lambda x: x[0], reverse=True)[:count]
            data = [i for i, j in sorted_data] + [j for i, j in sorted_data]

            top_df = pd.DataFrame(np.array(data).reshape(-1,len(data)), columns=column_names)
            #print(top_10_df)
            df = pd.concat([df, top_df], axis=0, ignore_index=True)
        except:
            top_df = pd.DataFrame(np.array([None for _ in range(count*2)]).reshape(-1,len(data)), columns=column_names)
            df = pd.concat([df, top_df], axis=0, ignore_index=True)
    return df
        


# get the top frequencies and the corresponding power value in separate cells
psd_df = pd.DataFrame()
for column in df.columns:
    if 'psd' in column:
        psd_df = pd.concat([psd_df, process_psd(df, column, 10)], axis=1)

df = pd.concat([df, psd_df], axis=1)

df.drop(['_time'], axis=1, inplace=True)
for column in df.columns:
    if 'psd' in column:
        df.drop(labels=column, axis=1, inplace=True)

### Dealing With Missing Data

Sometimes there can be missing values. These rows can be either deleted, although that can lead to inconsitency, or they can be estimated using different methods. In this case I used an Imputer from [scikit-learn](https://scikit-learn.org/stable/) to calculate the mean of the DataFrame column and fill in the missing values with the help of that.

In [5]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="mean")
for column in df.columns:
    if df[column].isna().any():
        df[column] = imputer.fit_transform(df[[column]])

If you want to inspect the transformed dataset, you can export it to *.csv* and view it in Excel

In [9]:
df.head()
#df.to_csv('output.csv')

### Scaling/Standardization

Standardization is an important step in preprocessing: many ML models need datasets that have features following a near Gaussian distribution with zero mean and unit variance. For this reason we can use the *StandardScaler* from scikit-learn to fit a scaler to our dataset.

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df)
df_scaled = scaler.transform(df)

print(df_scaled.mean(axis=0))

[-1.34395419e-16  1.05179023e-16  1.57768535e-16 -3.64620614e-15
  1.09853647e-14 -8.22295448e-15  1.75298372e-16 -7.94503352e-16
 -1.25849623e-15  1.15404762e-16  6.23770041e-16 -8.79413501e-16
  3.12615431e-16  5.12747739e-16 -1.25630500e-16  6.83663652e-16
  5.84327908e-17 -9.34924652e-17 -4.67462326e-17  2.16201326e-16
  7.88842675e-17  0.00000000e+00 -5.25895117e-17  5.84327908e-17
  5.84327908e-17 -1.16865582e-16  0.00000000e+00  5.25895117e-17
 -6.77820373e-16  1.24754008e-15  5.87249547e-16  9.86053344e-16
 -7.24566606e-16  4.35324291e-16  8.76491862e-17  1.58571986e-15
  5.27721142e-17  3.50596745e-17  5.84327908e-18  1.66533454e-16
 -4.67462326e-17 -5.84327908e-17  6.42760698e-17  6.42760698e-17
 -1.75298372e-17  0.00000000e+00  2.86320675e-16 -2.72881133e-15
  6.77820373e-16 -7.12880047e-16  2.07436407e-16 -1.17376868e-15
  8.47275466e-16  1.45935895e-15 -2.15032670e-15 -1.51925256e-16
  7.01193489e-17 -6.86585292e-17  5.84327908e-17 -5.11286919e-17
  8.18059071e-17 -1.22708

In [8]:
print(df_scaled.std(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
